This notebook allows browsing the fitted data, exporting an .html preview, and exporting all to .csv

In [ ]:
import numpy as np
import math
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1" # required to enable .exr for new opencv-python versions
import cv2
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"} # making matplotlib background white
from pathlib import Path
import sys
from tqdm.auto import tqdm
import pandas as pd
from IPython.display import Image
import ipywidgets

In [ ]:
outputs_path = Path("/projects/SkyGAN/clouds_fisheye/solar_disk_fitting/")

def get_photos_path(shooting_name):
    Path("/projects/SkyGAN/clouds_fisheye/processed/") / shooting_name / "1K_EXR"

## Show all fitted outputs in the notebook

In [ ]:
views_per_shooting = []

for shooting_path in sorted(outputs_path.iterdir()):
    if not shooting_path.is_dir() or not any(shooting_path.iterdir()):
        continue

    shooting_name = shooting_path.name

    views_per_shooting.append(ipywidgets.VBox([
        ipywidgets.HTML(value=f"<h2>{shooting_name}</h2>"),
        ipywidgets.HBox([
            ipywidgets.Image(value=open(shooting_path / "time_offset_fitting.png", 'rb').read(),
                format='png', height=250),
            ipywidgets.Image(value=open(shooting_path / "solar_disk_fitting.png", 'rb').read(),
                format='png', height=250)
        ])
    ]))

display(ipywidgets.VBox(views_per_shooting))

## Export all fitted outputs to .html

Note that the `.html` file simply links to `.png` plots and assumes they are in a subdirectory.

In [ ]:
html = '''<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>title</title>
  </head>
  <body>
'''

for shooting_path in sorted(outputs_path.iterdir()):
    if not shooting_path.is_dir() or not any(shooting_path.iterdir()):
        continue

    shooting_name = shooting_path.name

    html += f"<h2>{shooting_name}</h2>"
    html += f"<img style='width:40%' src='solar_disk_fitting/{shooting_name}/time_offset_fitting.png' />"
    html += f"<img style='width:40%' src='solar_disk_fitting/{shooting_name}/solar_disk_fitting.png' />"

html += '''
</body>
</html>'''

with open("solar_disk_fitting_20221111_1738.html", 'w') as f:
    f.write(html)

## Export all fitted outputs to .csv

In [ ]:
overall_results = {
    "t": [],
    "shoot": [],
    "sun_elevation": [],
    "sun_azimuth": [],
    "img_fname": [],
}

for shooting_path in sorted(outputs_path.iterdir()):
    if not shooting_path.is_dir() or not any(shooting_path.iterdir()):
        continue

    shooting_name = shooting_path.name
    shooting_results = np.load(shooting_path / "fitting_results.npz", allow_pickle=True)

    overall_results["t"].extend(shooting_results["t"])
    overall_results["shoot"].extend([shooting_name] * len(shooting_results["t"]))
    overall_results["sun_elevation"].extend(shooting_results["corrected_elevation"])
    overall_results["sun_azimuth"].extend(shooting_results["corrected_azimuth"])
    overall_results["img_fname"].extend(shooting_results["img_fname"])

overall_results_df = pd.DataFrame(overall_results)
overall_results_df.to_csv("solar_disk_fitting_20221111_1738.csv")